[View in Colaboratory](https://colab.research.google.com/github/Jeuni/dubluve/blob/master/Multi-Layered_Perceptron_Model.ipynb)

In [7]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
from google.colab import drive
drive.mount('/content/gdrive')

gpg: keybox '/tmp/tmpagks366t/pubring.gpg' created
gpg: /tmp/tmpagks366t/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Mounted at /content/gdrive


In [2]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install pandas

    100% |████████████████████████████████| 253.3MB 87kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x4f31c000 @  0x7f826f70e1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8


In [3]:
# To determine which version you're using:
!pip show tensorflow

# For the current version: 
!pip install --upgrade tensorflow

# For a specific version:
!pip install tensorflow==1.2

# For the latest nightly build:
!pip install tf-nightly

Name: tensorflow
Version: 1.10.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: astor, grpcio, protobuf, numpy, termcolor, wheel, gast, setuptools, tensorboard, absl-py, six
Required-by: 
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (1.10.1)
    100% |████████████████████████████████| 35.0MB 1.2MB/s 
    100% |████████████████████████████████| 890kB 6.6MB/s 
    100% |████████████████████████████████| 245kB 25.3MB/s 
  Running setup.py bdist_wheel for html5lib ... - \ done
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
  Running setup.py bdist_wheel for markdown ... - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/b6/52/17/f0af18e3e0ec6fa60b361ffed15b4c3468f6

In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


Redirecting output to ‘wget-log’.
Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
#Import
import numpy as np
import pandas as pd
import tensorflow as tf

In [12]:
# Import datasets & set datasets

X_data = list()
Y_data = list()
X_test = list()
Y_test = list()

def split(dataset, result, dx, dy):
    for row in dataset:
        rows = list()
        for column in row:
            rows.append(column.split())
        rowdata = [list(map(int, i)) for i in rows]
        dx.append(rowdata[0])
        dy.append(result)


def get(n_datasets, path, result, dx, dy):
    filename = input('Please type the name of file : ')
    for i in range(1, n_datasets):
        name = path + filename + '/' + filename + str(i) + '.csv'
        file = pd.read_csv(name)
        dataset = file.values.tolist()
        split(dataset, result, dx, dy)

def load(x, y, result):
  if str(result) == '[1, 0]':
      path = 'gdrive/My Drive/datasets/'
  else:
      path = 'gdrive/My Drive/datasets/nobody/'
  n_datasets = int(input('Please type the number of datasets : '))
  get(n_datasets, path, result, x, y)
  

n = int(input("Please type the number of days: "))
for i in range(n):
    load(X_data, Y_data, [1, 0])  # Exist
load(X_data, Y_data, [0, 1])  # No Exist
# Test Set
load(X_test, Y_test, [1, 0]) 
load(X_test, Y_test, [0, 1])



Please type the number of days: 5
Please type the number of datasets : 181
Please type the name of file : labeled_data1_20180814_153212_human
Please type the number of datasets : 127
Please type the name of file : labeled_data1_20180813_164819_human
Please type the number of datasets : 291
Please type the name of file : labeled_data1_20180810_125246_human
Please type the number of datasets : 156
Please type the name of file : labeled_data1_20180810_092138_human
Please type the number of datasets : 87
Please type the name of file : labeled_data1_20180809_173925_human
Please type the number of datasets : 59
Please type the name of file : labeled_data1_20180814_102745_none
Please type the number of datasets : 355
Please type the name of file : labeled_data1_20180814_102745_human
Please type the number of datasets : 18
Please type the name of file : labeled_data3_20180814_153212_none


In [0]:
learning_rate = 0.02
total_epochs = 500
log_path='gdrive/My Drive/test_logs0923_dnn_rate002_layermod_e1000'
save_path = 'gdrive/My Drive/model/dnn0923_rate002_layermod_e1000'

In [0]:
tf.reset_default_graph()
# Construct the model
global_step = tf.Variable(0, trainable=False, name='global_step')
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

In [0]:
with tf.name_scope('weight'):
  W1 = tf.Variable(tf.random_uniform([100, 100], -1., 1.), name='W1')
  W2 = tf.Variable(tf.random_uniform([100, 50], -1., 1.), name='W2')
  W3 = tf.Variable(tf.random_uniform([50, 25], -1., 1.), name='W3')
  W4 = tf.Variable(tf.random_uniform([25, 2], -1., 1.), name='W4')
  b1 = tf.Variable(tf.zeros([100]), name='b1')
  b2 = tf.Variable(tf.zeros([50]), name='b2')
  b3 = tf.Variable(tf.zeros([25]), name='b3')
  b4 = tf.Variable(tf.zeros([2]), name='b4')
  
  W_hist1 = tf.summary.histogram('Weight1', W1)
  W_hist2 = tf.summary.histogram('Weight2', W1)  
  W_hist3 = tf.summary.histogram('Weight3', W1)
  W_hist4 = tf.summary.histogram('Weight4', W1)
  b_hist1 = tf.summary.histogram('bias1', b1)
  b_hist2 = tf.summary.histogram('bias2', b1)  
  b_hist3 = tf.summary.histogram('bias3', b1)
  b_hist4 = tf.summary.histogram('bias4', b1)


In [0]:
with tf.name_scope('layer1'):
  L1 = tf.add(tf.matmul(X, W1), b1)
  L1 = tf.nn.relu(L1)
with tf.name_scope('layer2'):
  L2 = tf.add(tf.matmul(L1, W2), b2)
  L2 = tf.nn.relu(L2)
with tf.name_scope('layer3'):
  L3 = tf.add(tf.matmul(L2, W3), b3)
  L3 = tf.nn.relu(L3)
with tf.name_scope('layer4'):
  model = tf.add(tf.matmul(L3, W4), b4, name='model')

In [17]:
with tf.name_scope('cost'):
  # Cross-Entropy
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
  cost_summ = tf.summary.scalar('cost', cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [23]:
# Training

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(cost, global_step=global_step, name='op')

with tf.name_scope('accuracy'):
  is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
  accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

with tf.Session() as sess:  
  merged = tf.summary.merge_all()
  writer = tf.summary.FileWriter(log_path, sess.graph)
  sess = tf.Session()
  saver = tf.train.Saver(tf.global_variables())
  sess.run(tf.global_variables_initializer())
  
  # make input and target to train
  batch_xs = X_data
  batch_ys = Y_data 

for step in range(total_epochs):
  summary, _, loss =  sess.run([merged, train_op, cost], 
                      feed_dict={X: batch_xs, Y: batch_ys})
  writer.add_summary(summary, step)
  
  print('Step: %d, ' % sess.run(global_step), 
        'Cost: %.3f' % sess.run(cost, feed_dict={X: X_data, Y: Y_data}))

saver.save(sess, save_path, global_step=global_step)


Step: 1,  Cost: 326618.906
Step: 2,  Cost: 182144.938
Step: 3,  Cost: 85614.859
Step: 4,  Cost: 37781.051
Step: 5,  Cost: 26800.996
Step: 6,  Cost: 32245.660
Step: 7,  Cost: 18815.539
Step: 8,  Cost: 4764.039
Step: 9,  Cost: 13053.898
Step: 10,  Cost: 11220.648
Step: 11,  Cost: 2702.407
Step: 12,  Cost: 12992.347
Step: 13,  Cost: 14131.169
Step: 14,  Cost: 6122.830
Step: 15,  Cost: 7030.607
Step: 16,  Cost: 12129.751
Step: 17,  Cost: 11026.570
Step: 18,  Cost: 5012.800
Step: 19,  Cost: 5122.964
Step: 20,  Cost: 8577.138
Step: 21,  Cost: 5041.815
Step: 22,  Cost: 2948.223
Step: 23,  Cost: 5258.666
Step: 24,  Cost: 3463.521
Step: 25,  Cost: 1495.036
Step: 26,  Cost: 3109.349
Step: 27,  Cost: 958.986
Step: 28,  Cost: 3199.952
Step: 29,  Cost: 3569.375
Step: 30,  Cost: 1154.573
Step: 31,  Cost: 3648.904
Step: 32,  Cost: 4166.133
Step: 33,  Cost: 1072.820
Step: 34,  Cost: 3577.404
Step: 35,  Cost: 4876.316
Step: 36,  Cost: 3478.708
Step: 37,  Cost: 650.805
Step: 38,  Cost: 2659.602
Step: 39

'gdrive/My Drive/model/dnn0923_rate002_layermod_e1000-500'

In [19]:
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값: ', sess.run(prediction, feed_dict={X: X_data}))
print('실제값: ', sess.run(target, feed_dict={Y: Y_data}))

예측값:  [0 0 0 ... 1 1 1]
실제값:  [0 0 0 ... 1 1 1]


In [22]:
# Print the Accuracy

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: X_test, Y: Y_test}))

정확도: 76.58
